In [3]:
## Code for loading dataset to generate tables for blog (not part of blog post).
import pandas as pd
import numpy as np
df = pd.read_csv("data/EmployeeSatisfactionIndex.csv")
df = df.drop(columns=['emp_id', 
                      'location', 
                      'education', 
                      'Unnamed: 0']).loc[(df['Dept'] == 'HR') | (df['Dept'] == 'Sales') | (df['Dept'] == 'Purchasing')]
df['satisfied'] = df['satisfied'].replace({1: True, 0: False})
# df

In [4]:
# complete df - original (no missingness)
data = df.drop(columns=['rating', 'job_level', 'certifications', 'awards', 'onsite', 'salary','age'])

complete_data = pd.DataFrame(data.drop(columns=['recruitment_type']).groupby('Dept')['satisfied'].value_counts()).unstack().T.transform(lambda x: (x / x.sum()).round(2))

# show top 10 as output in blog 
data = data.tail(10).reset_index().drop(columns='index')
# complete_data

In [5]:
#Insert df with pattern missingness

# insert missing in sales dept
df_mar = df.copy()
df_mar.loc[((df_mar['Dept'] == "Sales") & (df_mar['recruitment_type'] == 'Recruitment Agency') | (df_mar['recruitment_type'] == 'On-Campus')),'satisfied'] = np.nan

df_mar['satisfied'] = df_mar['satisfied'].replace({1: True, 0: False})

# resulting DF percentages of responses (ignores the missing)
df_mar = df_mar.drop(columns=['rating', 'job_level', 'certifications', 'awards', 'onsite', 'salary','age', 'recruitment_type'])
missing_pattern = pd.DataFrame(df_mar.groupby('Dept')['satisfied'].value_counts()).unstack().T.transform(lambda x: (x / x.sum()).round(2))
# missing_pattern

In [6]:
# Insert df with random missingness

df_mcar = df.copy()
# num of missing cells to add (based on how many were added above)
miss_no = df.loc[((df['Dept'] == "Sales") & (df['recruitment_type'] == 'Recruitment Agency') | (df['recruitment_type'] == 'On-Campus'))].shape[0]

# insert random missing in satisfied column
rows_to_update_addl = df_mcar.sample(miss_no, random_state=150).index # more to match total NaN count
df_mcar.loc[rows_to_update_addl] = np.nan
df_mcar['satisfied'] = df_mcar['satisfied'].replace({1: True, 0: False})

# resulting DF percentages of responses (ignores the missing)
df_mcar = df_mcar.drop(columns=['rating', 'job_level', 'certifications', 'awards', 'onsite', 'salary','age', 'recruitment_type'])
missing_random = pd.DataFrame(df_mcar.groupby('Dept')['satisfied'].value_counts()).unstack().T.transform(lambda x: (x / x.sum()).round(2))
# missing_random


# data table with NA's
data_missing = df.drop(columns=['rating', 'job_level', 'certifications', 'awards', 'onsite', 'salary','age'])
data_missing.loc[495,'satisfied'] = 'NA'
data_missing.loc[490,'satisfied'] = 'NA'
data_missing

,Dept,recruitment_type,satisfied
0,HR,Referral,True
3,Sales,On-Campus,True
4,HR,Recruitment Agency,True
5,Purchasing,Referral,True
6,Purchasing,Referral,False
...,...,...,...
489,HR,On-Campus,True
490,Purchasing,Recruitment Agency,NA
492,HR,On-Campus,True
494,Sales,Referral,True


In [7]:
data.to_csv('data.csv')

In [8]:
complete_data.to_csv('complete_data.csv')

In [9]:
missing_random.to_csv('missing_random.csv')

In [10]:
missing_pattern.to_csv('missing_pattern.csv')